Computes Vanilla CSM model hadron yield ratios that are not normalized to the high-multiplicity limit. Also outputs (dNch/dy) and (dNch/dEta), under the assumption (dNch/dEta) = (dNch/dy) / 1.2 (see https://arxiv.org/abs/1908.02792)

In [1]:
#include "inc/loadFISTlibs.h"

###############################################################################
#                                                                             #
# This is Thermal-FIST version 1.2.2                                          #
#                                                                             #
# Copyright (c) 2019 Volodymyr Vovchenko <vovchenko@fias.uni-frankfurt.de>    #
#                                                                             #
# Distributed under the GNU General Public License 3.0 (GPLv3 or later)       #
#                                                                             #
# Please cite when using this code:                                           #
# V. Vovchenko, H. Stoecker, arXiv:1901.05249 [nucl-th]                       #
#                                                                             #
# The latest version is available at https://github.com/vlvovch/Thermal-FIST  #
#                                       

In [2]:
using namespace std;

In [3]:
using namespace thermalfist;

In [4]:
bool useWidth  = true;
bool useQStats = false;

double Tch = 0.155;

In [5]:
// Particle list
//ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list-withnuclei.dat");
ThermalParticleSystem parts(string(INPUT_FOLDER) + "/list/PDG2014/list.dat");

Fill the list of the considered hadron yield ratios

In [6]:
vector<int> pdgs1, pdgs2;
vector<string> names1, names2;

names1.push_back("d");
names2.push_back("p");
pdgs1.push_back(1000010020);
pdgs2.push_back(2212);

names1.push_back("He3");
names2.push_back("p");
pdgs1.push_back(1000020030);
pdgs2.push_back(2212);

names1.push_back("H3La");
names2.push_back("p");
pdgs1.push_back(1010010030);
pdgs2.push_back(2212);

names1.push_back("He4");
names2.push_back("p");
pdgs1.push_back(1000020040);
pdgs2.push_back(2212);

names1.push_back("H3La");
names2.push_back("Lambda");
pdgs1.push_back(1010010030);
pdgs2.push_back(3122);

Compute the limiting GCE yield ratios values

In [7]:
vector<double> ratiosGCE;
{
    ThermalModelIdeal modelgce(&parts);

    if (useWidth)
        modelgce.SetUseWidth(ThermalParticle::eBW);
    else
        modelgce.SetUseWidth(ThermalParticle::ZeroWidth);
    modelgce.SetStatistics(useQStats);
    // Include quantum statistics for mesons only
    if (useQStats){
        for (int i = 0; i < modelgce.TPS()->Particles().size(); ++i) {
            ThermalParticle &part = modelgce.TPS()->Particle(i);
            if (part.BaryonCharge() != 0)
                part.UseStatistics(false);
        }
    }

    modelgce.SetTemperature(Tch);
    modelgce.SetBaryonChemicalPotential(0.);
    modelgce.SetElectricChemicalPotential(0.);
    modelgce.SetStrangenessChemicalPotential(0.);

    modelgce.FillChemicalPotentials();

    modelgce.CalculateDensities();

    for (int i = 0; i < names1.size(); ++i) {
        ratiosGCE.push_back(modelgce.GetDensity(pdgs1[i], 1) / modelgce.GetDensity(pdgs2[i], 1));
    }
}

Prepare the output (on the screen and to a file)

In [8]:
vector<std::ostream*> outs;
outs.push_back(&std::cout);

outs.push_back(NULL);

Routine performing the (logarithmic scale) scan over Vc and dNpi/dy.

Parameter k determines Vc = k dV/dy

In [9]:
void PerformScan(ThermalModelBase *model, 
                   double k,
                   const string& filename)
{
    ofstream fout(filename);
    outs[1] = &fout;
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "dNpipm/dy";
        *out << setw(15) << "Vc[fm3]";
        *out << setw(15) << "dNch/dy";
        *out << setw(15) << "dNch/deta";

        for (int i = 0; i < names1.size(); ++i) {
            *out << setw(15) << (names1[i] + "/" + names2[i]) + "";
        }
        
        *out << setw(15) << "S3";

        *out << endl;
    }
    
    double Vmin = 1. * k;
    double Vmax = 15000. * k;
    int iters = 30;
    double logVmin = log10(Vmin), logVmax = log10(Vmax);
    double dlogV = (logVmax - logVmin) / (iters - 1);
    
    double dNchdyOverdNchddeta = 1.20; // pp value from https://arxiv.org/abs/1908.02792
    
    for (double logV = logVmin; logV <= logVmax + 1.e-3; logV += dlogV) {
        double V = pow(10., logV);

        model->SetVolume(V);
        model->SetCanonicalVolume(V);
        model->CalculateDensities();

        for(std::ostream* out : outs) {
            *out << setw(15) << 2. * model->GetDensity(211, 1) * V / k; // Charged pions dN/dy
            *out << setw(15) << V; // Vc
            *out << setw(15) << model->ChargedMultiplicityFinal(0) / k;
            *out << setw(15) << model->ChargedMultiplicityFinal(0) / k / dNchdyOverdNchddeta;

            for (int i = 0; i < names1.size(); ++i) {
                *out << setw(15) << model->GetDensity(pdgs1[i], 1) / model->GetDensity(pdgs2[i], 1);
            }
            
            *out << setw(15) << (model->GetDensity(1010010030, 1) / model->GetDensity(1000020030, 1)) / (model->GetDensity(3122, 1) / model->GetDensity(2212, 1));

            *out << endl;
        }
    }
    
    for(std::ostream* out : outs) {
        *out << setw(15) << "HML";
        *out << setw(15) << "HML";
        *out << setw(15) << "HML";
        *out << setw(15) << "HML";

        for (int i = 0; i < ratiosGCE.size(); ++i) {
            *out << setw(15) << ratiosGCE[i];
        }
        
        *out << setw(15) << ratiosGCE[4] / ratiosGCE[1];

        *out << endl;
    }
    
    fout.close();
}

Perform the vanilla CSM scans

In [10]:
ThermalModelCanonical modelce(&parts);

if (useWidth)
    modelce.SetUseWidth(ThermalParticle::eBW);
else
    modelce.SetUseWidth(ThermalParticle::ZeroWidth);

modelce.SetStatistics(useQStats);
// Include quantum statistics for mesons only
if (useQStats){
    for (int i = 0; i < modelce.TPS()->Particles().size(); ++i) {
        ThermalParticle &part = modelce.TPS()->Particle(i);
        if (part.BaryonCharge() != 0)
            part.UseStatistics(false);
    }
}

modelce.SetTemperature(Tch);
modelce.SetBaryonChemicalPotential(0.);
modelce.SetElectricChemicalPotential(0.);
modelce.SetStrangenessChemicalPotential(0.);

modelce.SetBaryonCharge(0);
modelce.SetElectricCharge(0);
modelce.SetStrangeness(0);

modelce.FillChemicalPotentials();

In [11]:
PerformScan(&modelce, 1., "out/VanillaCSM.S3.Vc.eq.dVdy.dat");

      dNpipm/dy        Vc[fm3]        dNch/dy      dNch/deta            d/p          He3/p         H3La/p          He4/p    H3La/Lambda             S3
      0.0766118              1      0.0814348      0.0678623    4.39316e-05    7.18969e-10    2.74001e-10    1.04936e-14    7.08679e-10       0.985687
       0.111174        1.39317       0.118425      0.0986874     5.3271e-05    1.07856e-09    4.52602e-10    1.83853e-14    1.15673e-09        1.07247
       0.163537        1.94091       0.174725       0.145604    6.65589e-05    1.72585e-09     7.7999e-10    3.51208e-14    1.96128e-09        1.13641
        0.24466        2.70402       0.262495       0.218746    8.55916e-05    2.93138e-09    1.39649e-09    7.33834e-14    3.43527e-09        1.17189
       0.373351        3.76715       0.402927       0.335772    0.000113073    5.25294e-09    2.58693e-09    1.67317e-13    6.18126e-09        1.17672
       0.581907        5.24826       0.633185       0.527654    0.000153105    9.87053e-09    

In [12]:
PerformScan(&modelce, 3., "out/VanillaCSM.S3.Vc.eq.3dVdy.dat");

      dNpipm/dy        Vc[fm3]        dNch/dy      dNch/deta            d/p          He3/p         H3La/p          He4/p    H3La/Lambda             S3
      0.0928902              3      0.0998243      0.0831869     9.3132e-05    3.49993e-09    1.68812e-09    9.41377e-14    4.11846e-09        1.17673
        0.14269         4.1795       0.154346       0.128622     0.00012402    6.36975e-09    3.15818e-09    2.20307e-13    7.46512e-09        1.17196
       0.223818        5.82274       0.244318       0.203598    0.000169141    1.21337e-08    6.08185e-09     5.5709e-13     1.3807e-08        1.13791
       0.356972        8.11205       0.394594       0.328828    0.000235558    2.40468e-08     1.2002e-08    1.50615e-12    2.59394e-08         1.0787
       0.572334        11.3014        0.64335       0.536125    0.000333638    4.92144e-08    2.41157e-08    4.29739e-12    4.93003e-08        1.00175
       0.906276        15.7448        1.04026        0.86688    0.000477543    1.02861e-07    

In [13]:
PerformScan(&modelce, 6., "out/VanillaCSM.S3.Vc.eq.6dVdy.dat");

      dNpipm/dy        Vc[fm3]        dNch/dy      dNch/deta            d/p          He3/p         H3La/p          He4/p    H3La/Lambda             S3
       0.116668              6       0.127479       0.106233    0.000174143    1.28882e-08    6.46148e-09    6.07955e-13    1.46088e-08         1.1335
       0.186274          8.359       0.206181       0.171818    0.000242942    2.56254e-08    1.27758e-08    1.65266e-12     2.7479e-08        1.07234
       0.298551        11.6455       0.336178       0.280148    0.000344528     5.2573e-08    2.57016e-08    4.73479e-12     5.2271e-08       0.994256
       0.471795        16.2241       0.542654       0.452212    0.000493389    1.10009e-07    5.20607e-08    1.40455e-11    1.00117e-07       0.910081
       0.724597        22.6029       0.853019       0.710849    0.000706339    2.30726e-07    1.04589e-07    4.21164e-11    1.91825e-07       0.831399
        1.08024        31.4896        1.29944        1.08287    0.000997334    4.73986e-07    